In [1]:
import pandas as pd
import snowflake.connector

In [2]:
#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [6]:
#Get work logs
sql = f'''
select
pvid p_vid,
processed_attributes attributes
from
productvariants
order by random()
limit 20000
'''
cs.execute(sql)
df = cs.fetch_pandas_all()


In [7]:
def category(attributes):
    try:
        cat = json.loads(attributes)['fb_product_category']
        return cat
    except Exception:
        return None
    
df['CATEGORY'] = df['ATTRIBUTES'].apply(category)

mask = df['CATEGORY'].isnull()
df = df[~mask]

global exclude
exclude = [
    'availability','title','breadcrumbs','options','link','description','bullets','image_links','videos','currency','brand','brand',
    'description_structured','sku','key_value_pairs','key_value_pairs','fb_product_category','variant_matchable_id','item_group_id',
    'job_id','id','real_price','parent_website_url'
]

df['ATTRIBUTES'] = df['ATTRIBUTES'].apply(lambda x: list(set([x for x,k in json.loads(x).items() if x not in exclude])))
df = df.explode('ATTRIBUTES')
df = df.groupby(['CATEGORY','ATTRIBUTES'],as_index=False).count().rename(columns={'P_VID':'T_CATEGORY'})

df = df.pivot(index='CATEGORY', columns='ATTRIBUTES', values='T_CATEGORY').reset_index()
col_list= df.iloc[:,1:].columns
df['TOTAL'] = df[col_list].sum(axis=1)

dataframe_to_result(result,df.sort_values('TOTAL',ascending=False).iloc[0:1000])


NameError: name 'dataframe_to_result' is not defined